In [ ]:
%load_ext autoreload

%autoreload 2

from lentil import models
from lentil import evaluate
from lentil import datatools


import evaluate_experiment as e
import pandas as pd
import numpy as np
import pickle
import math
import sqlite3
from collections import defaultdict, namedtuple
from collections import Counter

Instance = namedtuple('Instance', 'user item time_elapsed outcome'.split())

In [ ]:
def convertIntoList(data, feature, avg=False, sort=False, unique=False, differences=False, fl = False):
    lol = 0
    features = []
    for i in range(len(list(data[feature]))):
        try:
            if fl==False:
                features.append(int(list(data[feature])[i]))
            else:
                features.append(float(list(data[feature])[i]))
        except:
            lol += 1
    if unique:
        features = set(features)
    if sort:
        features = sorted(features)
    if differences:
        features = [sorted(set(features))[i+1]-sorted(set(features))[i] for i in range(len(sorted(set(features)))-1)]
    if avg:
        features = float(sum(features))/len(features)
    return features

In [312]:
import_cols = ['Anon Student Id', 'Session Id', 'Problem Name', 'Time Elapsed', 'Outcome', 'Condition Name', 'Condition Type', 'Duration', 'Time', 'Step Name', 'Selection', 'Level (Unit)', 'Problem View', 'KC (Default)']
#import_cols = ['user', 'item', 'time_elapsed', 'outcome', 'schedulers', 'introduced', 'durations', 'timestamps']

In [318]:
japitems =['Environment','Earth','To_protect','Atomic_energy','To_proceed','Lifespan','Warming','Being_left_out','To_commit_suicide','To_lose','Ultraviolet','Exhaust_gas','Air','Conservation','Rise_in_temperature','Co2','Ageing_society','Sun','To_get_on_well','Average','Lonely','Accounting','Lack','Economics','Company_employee','Declining_birth_rate','Tree_dies','To_get_a_license','To_be_healed','Pension','To_disappear','Value','Wind-force','Solitude','Purpose','Global_warming','International_conference','Solar_heat','Raise_children','Relation_to','National_resource','Hard_rubbish','Disposable','Quarrel','Stay_indoors','To_not_get_on_well','Occupation','Hydrolytic_power','Bullying','To_associate_with','Generation_of_electricity','Be_polluted','To_save','Oil','Psychology','Water_shortage','Waste','Problem','To_look_after','Government']


In [326]:
def getDataOfUser(uid, intr=False, file=None, ts=False, out=False,sched=1,exper=1,strat=1):
    histories = []
    time_histories = []
    users = []
    items = []
    schedulers = []
    introduced = []
    durations = []
    timestamps = []

    conn = sqlite3.connect(file)
    c = conn.cursor()

    for row in c.execute("SELECT rowid, * FROM flashcard ORDER BY id"):
        if row[6] == uid:
            schedulers.append(row[-7])
            histories.append(row[-11])
            time_histories.append(row[-10])
            items.append(row[1])
            users.append(row[6])
            introduced.append(row[-3])
            durations.append(row[-8])
            timestamps.append(row[-9])

    history = [[str(y) for y in x] for x in [str(i).split(',') for i in histories]]
    time_history = [[str(y) for y in x] for x in [str(i).split(',') for i in time_histories]]
    item = range(1,61)
    user = [int(i) for i in users]
    scheduler = [int(i) for i in schedulers]
    introduce = [i for i in introduced]
    duration = [[str(y) for y in x] for x in [str(i).split(',') for i in durations]]
    timestamp = [[str(y) for y in x] for x in [str(i).split(',') for i in timestamps]]

    
    data = pd.DataFrame(columns=import_cols)
    intro = []
    tstamp = []
    outc = []
    condition_name = ['efc', 'leitner','flashcard']
    outcomee = ['INCORRECT', 'CORRECT']
    
    for y in range(uid-1, uid):
        #print str(y) + ' out of ' + str(len(set(user)))
        for x in range(len(item)):
            problem_view = 1
            if scheduler[x] == sched:
                intro.append(introduce[x])
                tstamp.append(timestamp[x])
                outc.append(history[x])
                
            for i in range(len(history[x])):
                if history[x][i] != '':
                    temp = pd.DataFrame([[str(exper)+'_'+str(user[y]), 1, japitems[item[x]-1], time_history[x][i], outcomee[int(history[x][i])], condition_name[int(scheduler[x])-1]+'_'+str(strat), str(scheduler[x])+"_"+str(strat), duration[x][i], timestamp[x][i], 1, 1, 12,problem_view,japitems[item[x]-1]]],columns=import_cols)
                #temp = pd.DataFrame([[user[y], item[x], time_history[x][i], history[x][i], scheduler[x], introduce[x], duration[x][i], timestamp[x][i]]],columns=cols)
                    data = data.append(temp)
                problem_view += 1
                
    if intr:
        return intro
    elif ts:
        return tstamp
    elif out:
        return outc
    else: 
        return data

In [ ]:
sched1history = []
def getTimeHistory(sched_id, leng, control):
    data = []
    if sched_id == 1:
        for x in range(len(strategy_1)):
            if x == 0: 
                file = experiment1file
            elif x == 1:
                file = experiment2file
            else:
                file = experiment3file
            conn = sqlite3.connect(file)
            c = conn.cursor()
            
            for y in range(len(strategy_1[x])):
                for row in c.execute("SELECT rowid, * FROM flashcard ORDER BY id"):
                    if row[8] != '' and row[6] == strategy_1[x][y] and len([int(i) for i in str(row[8]).split(',')]) >= leng and row[-7] == control:
                        data.append([int(i) for i in str(row[8]).split(',')])
                        
    elif sched_id == 2:
        for x in range(len(strategy_2)):
            if x == 0: 
                file = experiment1file
            elif x == 1:
                file = experiment2file
            else:
                file = experiment3file
            conn = sqlite3.connect(file)
            c = conn.cursor()
            
            for y in range(len(strategy_2[x])):
                for row in c.execute("SELECT rowid, * FROM flashcard ORDER BY id"):
                    if row[8] != '' and row[6] == strategy_2[x][y] and len([int(i) for i in str(row[8]).split(',')]) >= leng and row[-7] == control:
                        data.append([int(i) for i in str(row[8]).split(',')])
    
    return data

sched1history = getTimeHistory(1, 4, 1)
sched1history = [sum(e)/len(e) for e in zip(*sched1history)]
sched1history.append([sum(e)/len(e) for e in zip(*getTimeHistory(1, 4, 1))][3])
sched1history.append([sum(e)/len(e) for e in zip(*getTimeHistory(1, 5, 1))][4])
print sched1history

sched1_3history = getTimeHistory(1, 4, 3)
sched1_3history = [sum(e)/len(e) for e in zip(*sched1_3history)]
print sched1_3history

sched2history = getTimeHistory(2, 3, 2)
sched2history = [sum(e)/len(e) for e in zip(*sched2history)]
sched2history.append([sum(e)/len(e) for e in zip(*getTimeHistory(2, 4, 2))][3])
sched2history.append([sum(e)/len(e) for e in zip(*getTimeHistory(2, 5, 2))][4])
print sched2history

sched2_3history = getTimeHistory(2, 3, 3)
sched2_3history = [sum(e)/len(e) for e in zip(*sched2_3history)]
print sched2_3history


In [ ]:
def getFeedbackFromUser(uid, file):
    conn = sqlite3.connect(file)
    c = conn.cursor()

    for row in c.execute("SELECT rowid, * FROM users ORDER BY id"):
        if row[11]=='3,2':
            print row[3], row[-6], row[-4]
            print ''
        
getFeedbackFromUser(1, experiment2file)

In [300]:
def getData(strategy_one, strategy_two, file, exper):
    one_data = pd.DataFrame(columns = import_cols)

    for i in range(len(strategy_one)):
        data = getDataOfUser(strategy_one[i], file=file, exper=exper, strat=1)
        one_data = one_data.append(data)

    two_data = pd.DataFrame(columns = import_cols)

    for i in range(len(strategy_two)):
        data = getDataOfUser(strategy_two[i], file=file, exper=exper, strat=2)
        two_data = two_data.append(data)
        
    return one_data, two_data

Begin

In [ ]:
experiment1file = 'classroom_data/actual/experiment1.sqlite'
experiment2file = 'classroom_data/actual/experiment2.sqlite'
experiment3file = 'classroom_data/actual/experiment3.sqlite'

experiment_files = [experiment1file, experiment2file, experiment3file]

In [320]:
strategy_1 = [[1,2,3,4,6], [12,13,14,15,17], [1,2,3,5]]
strategy_2 = [[11,12,13,14,16,17], [19,20,21,22,23,24,25,26], [7,8,9]]

In [327]:
data_1_1, data_1_2 = getData(strategy_1[0], strategy_2[0], experiment1file,1)

In [328]:
data_2_1, data_2_2 = getData(strategy_1[1], strategy_2[1], experiment2file,2)

In [329]:
data_3_1, data_3_2 = getData(strategy_1[2], strategy_2[2], experiment3file,3)

In [295]:
data_1 = data_2_1
data_2 = data_2_2

In [330]:
frames_1 = [data_1_1, data_2_1, data_3_1]
frames_2 = [data_1_2, data_2_2, data_3_2]
data_1 = pd.concat(frames_1)
data_2 = pd.concat(frames_2)

frames_3 = [data_1,data_2]
data = pd.concat(frames_3)

In [ ]:
one_data_strat = data_1[data_1['schedulers'] == 1.0]
one_data_control = data_1[data_1['schedulers'] == 3.0]
two_data_strat = data_2[data_2['schedulers'] == 2.0]
two_data_control = data_2[data_2['schedulers'] == 3.0]

In [ ]:
one_data_strat.to_csv('one_data_strat.csv')
one_data_control.to_csv('one_data_control.csv')
two_data_strat.to_csv('two_data_strat.csv')
two_data_control.to_csv('two_data_control.csv')

In [331]:
data.replace('', np.nan, inplace=True)
data = data.dropna()
data.to_csv('data.csv')

Durations

In [ ]:
def int_or_float(s):
    try:
        return int(s)
    except ValueError:
        return float(s)

def getDurations(sched, outcome):
    new_sched = sched.replace('', np.nan).dropna()
    if outcome != None:
        filtered = new_sched['outcome'] == str(outcome)
        new = new_sched[filtered]
    else:
        new = new_sched
    durations = [int_or_float(i) for i in list(new['durations'])]

    avg_duration = float(sum(durations))/float(len(durations))
    return avg_duration

print getDurations(one_data_strat, 1), getDurations(one_data_strat, 0)
print getDurations(one_data_control, 1), getDurations(one_data_control, 0)
print getDurations(two_data_strat, 1), getDurations(two_data_strat, 0)-0.2
print getDurations(two_data_control, 1), getDurations(two_data_control, 0)-0.2

Mean proportion correct

In [ ]:
outcome_1_1 = convertIntoList(one_data_strat, 'outcome')
outcome_1_3 = convertIntoList(one_data_control, 'outcome')

outcome_2_1 = convertIntoList(two_data_strat, 'outcome')
outcome_2_3 = convertIntoList(two_data_control, 'outcome')

print Counter(outcome_1_1)[1], Counter(outcome_1_3)[1]
print Counter(outcome_2_1)[1], Counter(outcome_2_3)[1]

In [ ]:
one_data_strat = []

for x in range(len(strategy_1)):
    for i in range(len(strategy_1[x])):
        data = getDataOfUser(strategy_1[x][i], file=experiment_files[x], out=True, sched=1)
        one_data_strat.append(data)
        data = []
    
one_data_control = []

for x in range(len(strategy_1)):
    for i in range(len(strategy_1[x])):
        data = getDataOfUser(strategy_1[x][i], file=experiment_files[x], out=True, sched=3)
        one_data_control.append(data)
        data = []
    
two_data_strat = []

for x in range(len(strategy_2)):
    for i in range(len(strategy_2[x])):
        data = getDataOfUser(strategy_2[x][i], file=experiment_files[x], out=True, sched=2)
        two_data_strat.append(data)
        data = []
    
two_data_control = []

for x in range(len(strategy_2)):
    for i in range(len(strategy_2[x])):
        data = getDataOfUser(strategy_2[x][i], file=experiment_files[x], out=True, sched=3)
        two_data_control.append(data)
        data = []

In [ ]:
def getNumberCorrect(scheduler):
    data = []
    final = []
    for y in range(len(scheduler)):
        for i in range(len(scheduler[y])):
            for x in range(len(scheduler[y][i])):
                if scheduler[y][i][x] != '':
                    data.append(int(scheduler[y][i][x]))
        total = float(sum(data))/float(len(data))
        final.append(total)
        data = []
    
    return final

In [ ]:
one_data_strat_correct = getNumberCorrect(one_data_strat)
one_data_control_correct = getNumberCorrect(one_data_control)
two_data_strat_correct = getNumberCorrect(two_data_strat)
two_data_control_correct = getNumberCorrect(two_data_control)

In [ ]:
print len(one_data_strat_correct)

In [ ]:
print sum(one_data_strat_correct)/len(one_data_strat_correct), sum(one_data_control_correct)/len(one_data_control_correct)
print sum(two_data_strat_correct)/len(two_data_strat_correct), sum(two_data_control_correct)/len(two_data_control_correct)

Number of words introduced

In [ ]:
one_data = []

for i in range(len(strategy_1)):
    data = getDataOfUser(strategy_1[i], intr=True)
    one_data.append(data)
    
two_data = []

for i in range(len(strategy_2)):
    data = getDataOfUser(strategy_2[i], intr=True)
    two_data.append(data)

In [ ]:
from collections import Counter 

introduced_1 = []
for i in range(len(strategy_1)):
    introduced_1.append(Counter(one_data[i])[''])
    
#introduced_1 = sum(introduced_1)/len(introduced_1)

introduced_2 = []
for i in range(len(strategy_2)):
    introduced_2.append(Counter(two_data[i])[''])

#introduced_2 = sum(introduced_2)/len(introduced_2)

print [30-i for i in introduced_1]
print [30-i for i in introduced_2]

In [ ]:
e.plotGraph([ ['Control', 'Leitner', 'EFC'], ['Control', 'Leitner', 'EFC'] ], [ [47.619047619, 56.3395581239, 70.6349206349], [34.76190477,37.9078197,50.55195913] ], ['Learning Accuracy', 'Retention Accuracy'])

Time elapsed

In [ ]:
time_elapsed_1_1 = convertIntoList(one_data_strat, 'time_elapsed', True)
time_elapsed_1_3 = convertIntoList(one_data_control, 'time_elapsed', True)

time_elapsed_2_1 = convertIntoList(two_data_strat, 'time_elapsed', True)
time_elapsed_2_3 = convertIntoList(two_data_control, 'time_elapsed', True)

print time_elapsed_1_1, time_elapsed_1_3
print time_elapsed_2_1, time_elapsed_2_3

Overall time taken

In [ ]:
one_data_strat = []

for x in range(len(strategy_1)):
    for i in range(len(strategy_1[x])):
        data = getDataOfUser(strategy_1[x][i], file=experiment_files[x], ts=True, sched=1)
        one_data_strat.append(data)
        data = []
    
one_data_control = []

for x in range(len(strategy_1)):
    for i in range(len(strategy_1[x])):
        data = getDataOfUser(strategy_1[x][i], file=experiment_files[x], ts=True, sched=3)
        one_data_control.append(data)
        data = []
    
two_data_strat = []

for x in range(len(strategy_2)):
    for i in range(len(strategy_2[x])):
        data = getDataOfUser(strategy_2[x][i], file=experiment_files[x], ts=True, sched=2)
        two_data_strat.append(data)
        data = []
    
two_data_control = []

for x in range(len(strategy_2)):
    for i in range(len(strategy_2[x])):
        data = getDataOfUser(strategy_2[x][i], file=experiment_files[x], ts=True, sched=3)
        two_data_control.append(data)
        data = []

In [ ]:
def getOverallTime(scheduler):
    data = []
    final = []
    for y in range(len(scheduler)):
        for i in range(len(scheduler[y])):
            for x in range(len(scheduler[y][i])):
                if scheduler[y][i][x] != '':
                    data.append(int(scheduler[y][i][x]))
        time = round((max(data) - min(data))/60.0,1)
        final.append(time)
        data = []
    
    return final

one_data_strat_time = getOverallTime(one_data_strat)
one_data_control_time = getOverallTime(one_data_control)
two_data_strat_time = getOverallTime(two_data_strat)
two_data_control_time = getOverallTime(two_data_control)

In [ ]:
print one_data_strat_time, one_data_control_time
print two_data_strat_time, two_data_control_time

Mean time history

In [ ]:
def getMeanTimeHistory(scheduler, length):
    items = list(scheduler['item'])
    timestamps = list(scheduler['time_elapsed'])
    p = list(zip(items, timestamps))
    d = {}
    for item, timestamp in p:
        key = item
        if key in d:
            v = d[key]
            v[0] += ", {}".format(str(timestamp))
        else:
            d[key] = [timestamp]
    
    x = []
    q = list(d.values())
    for i in range(len(q)):
        if q[i][0] != '':
            x.append(q[i])
    y = []
    dummy = 0
    for i in range(len(x)):
        try:
            y.append(x[i][0].split(','))
            y[i] = [int(o) for o in y[i]]
        except:
            y.pop()
            y.append([0])
            dummy += 1

    final = []
    for i in range(len(y)):
        if len(y[i]) > length:
            final.append(y[i])
    
    output = []
    for k in range(length):
        output.append(sum(final[k])/len(final[k]))
    return output

l = 3

print getMeanTimeHistory(one_data_strat, l), getMeanTimeHistory(one_data_control, l)
print getMeanTimeHistory(two_data_strat, l), getMeanTimeHistory(two_data_control, l)

Mean introduced seconds

In [ ]:
introduced_1_1 = convertIntoList(one_data_strat, 'introduced', True, True, True, True)
introduced_1_3 = convertIntoList(one_data_control, 'introduced', True, True, True, True)

introduced_2_1 = convertIntoList(two_data_strat, 'introduced', True, True, True, True)
introduced_2_3 = convertIntoList(two_data_control, 'introduced', True, True, True, True)

print introduced_1_1,introduced_1_3
print introduced_2_1,introduced_2_3

Durations

In [ ]:
durations_1_1 = convertIntoList(one_data_strat, 'durations', avg=True, fl=True)
durations_1_3 = convertIntoList(one_data_control, 'durations', avg=True, fl=True)

durations_2_1 = convertIntoList(two_data_strat, 'durations', avg=True, fl=True)
durations_2_3 = convertIntoList(two_data_control, 'durations', avg=True, fl=True)

print durations_1_1, durations_1_3
print durations_2_1, durations_2_3

Learning efficiency

In [ ]:
le_one_strategy = [a_i/b_i for a_i, b_i in zip(one_data_strat_correct, one_data_strat_time)]
le_one_control = [a_i/b_i for a_i, b_i in zip(one_data_control_correct, one_data_control_time)]
le_two_strategy = [a_i/b_i for a_i, b_i in zip(two_data_strat_correct, two_data_strat_time)]
le_two_control = [a_i/b_i for a_i, b_i in zip(two_data_control_correct, two_data_control_time)]

print le_two_strategy

le_one_strategy = sum(le_one_strategy)/len(le_one_strategy)
le_one_control = sum(le_one_control)/len(le_one_control)
le_two_strategy = sum(le_two_strategy)/len(le_two_strategy)
le_two_control = sum(le_two_control)/len(le_two_control)

print le_one_strategy, le_one_control
print le_two_strategy, le_two_control

Feedback